---

| | |
|---|---|
| **Autor notatnika** | Damian Sałkowski |
| **Wersja** | 1.0 |
| **Wydawca** | [sensai.academy](https://sensai.academy/) |
| **Lekcja** | Pozyskiwanie danych dla modeli językowych |
| **Tydzień programu** | 2 |
| **Pytania? Napisz na discord. 📩 ** | **damian_17365** |


---

#**❗️ Ważne: Zanim zaczniesz, zrób kopię tego notatnika!**

Aby móc edytować, uruchamiać kod i zapisywać zmiany, musisz pracować na własnej kopii tego pliku.

**Jak skopiować notatnik w Google Colab:**
1.  Przejdź do menu na górze strony.
2.  Kliknij **`Plik`**.
3.  Z rozwijanego menu wybierz opcję **`Zapisz kopię na Dysku`**.
4.  Nowa karta otworzy się z Twoją osobistą kopią notatnika (nazwa pliku będzie zaczynać się od "Kopia notatnika...").
5.  **Wszystkie dalsze kroki wykonuj w tej nowo otwartej kopii.**

## 1. Konfiguracja Kluczy API

Konfigurujemy niezbędne klucze API, których będziemy używać w kolejnych komórkach tego Colaba.

**Instrukcja dodawania kluczy do Colab Secrets:**

1.  Po lewej stronie interfejsu Colab, znajdź ikonę "klucza" (🔑 Sekrety), kliknij ją.
2.  Kliknij "Dodaj nowy sekret".
3.  W polu "Nazwa" wpisz dokładnie jedną z nazw kluczy użytych w skrypcie poniżej (np. `SECRET_OPENAI_API_KEY`).
4.  W polu "Wartość" wklej swój klucz API skopiowany z panelu dostawcy usługi.
5.  Upewnij się, że przełącznik "Notebook access" jest włączony dla tego sekretu.
6.  Powtórz dla każdego klucza, którego chcesz użyć.

Skrypt poniżej odczyta klucze z Colab Secrets i ustawi je jako zmienne środowiskowe, które biblioteki API automatycznie wykryją.

In [8]:
# Ważne: Klucze API najlepiej przechowywać w bezpieczny sposób, np. w Google Colab Secrets.
# Po lewej stronie interfejsu Colab, znajdź ikonę "klucza" (Sekrety), kliknij ją, a następnie
# dodaj nowe sekrety o nazwach zgodnych z poniższymi zmiennymi (np. SECRET_OPENAI_API_KEY).
# Wartości kluczy skopiuj z panelu dostawcy usługi.
# Ta komórka pobierze klucze z Colab Secrets i ustawi je jako zmienne środowiskowe.
# Jeśli nie masz klucza dla danej usługi, możesz pominąć dodawanie go do sekretów i jego pole zostanie puste.

import os
from google.colab import userdata

#@markdown Klucz OpenAI: Znajdź na https://platform.openai.com/account/api-keys
OPENAI_API_KEY = userdata.get('SECRET_OPENAI_API_KEY') #@param {type:"string"}
#@markdown Klucz Jina AI: Znajdź na https://jina.ai/reader/
JINA_API_KEY = userdata.get('SECRET_JINA_API_KEY') #@param {type:"string"}
#@markdown Klucz OpenRouter: Znajdź na https://openrouter.ai/settings/keys
OPENROUTER_API_KEY = userdata.get('SECRET_OPENROUTER_API_KEY') #@param {type:"string"}
#@markdown Klucz SerpData: Znajdź na https://serpdata.io/
SERPDATA_API_KEY = userdata.get('SECRET_SERPDATA_API_KEY') #@param {type:"string"}


# Ustawienie kluczy jako zmiennych środowiskowych
# Biblioteki klienckie (SDK) często automatycznie szukają klucza w zmiennych środowiskowych.
if OPENAI_API_KEY:
  os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
  print("OpenAI API Key załadowany ze zmiennych środowiskowych.")
else:
  print("Brak OpenAI API Key w Colab Secrets.")

if JINA_API_KEY:
  os.environ['JINA_API_KEY'] = JINA_API_KEY
  print("Jina AI API Key załadowany ze zmiennych środowiskowych.")
else:
  print("Brak Jina AI API Key w Colab Secrets.")

if OPENROUTER_API_KEY:
  os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY
  print("OpenRouter API Key załadowany ze zmiennych środowiskowych.")
else:
  print("Brak OpenRouter API Key w Colab Secrets.")

if SERPDATA_API_KEY:
  os.environ['SERPDATA_API_KEY'] = SERPDATA_API_KEY
  print("SERPDATA API Key załadowany ze zmiennych środowiskowych.")
else:
  print("Brak OpenRouter API Key w Colab Secrets.")


print("\nStatus ładowania kluczy API sprawdzony. Pamiętaj, że klucze są dostępne tylko dla tej sesji Colab.")

OpenAI API Key załadowany ze zmiennych środowiskowych.
Jina AI API Key załadowany ze zmiennych środowiskowych.
OpenRouter API Key załadowany ze zmiennych środowiskowych.
SERPDATA API Key załadowany ze zmiennych środowiskowych.

Status ładowania kluczy API sprawdzony. Pamiętaj, że klucze są dostępne tylko dla tej sesji Colab.


# 1. Dlaczego Potrzebujemy Danych Zewnętrznych? Ograniczenia Modeli Językowych

Modele językowe (LLM), takie jak GPT-4o czy Gemini, uczą się na podstawie danych dostępnych do pewnego momentu w przeszłości. Mają przez to kilka ograniczeń:

1.  **"Knowledge Cut-off" (Granica Wiedzy):** Trening modeli kończy się w określonym punkcie czasowym. Oznacza to, że **nie znają one wydarzeń ani informacji, które pojawiły się po tej dacie**.
2.  **Brak Dostępu do Danych Bieżących:** Standardowo modele **nie mają połączenia z internetem** i nie mogą sprawdzać informacji w czasie rzeczywistym (np. aktualnej pogody, wyników meczów na żywo).
3.  **Ryzyko Halucynacji:** Kiedy model jest pytany o coś, czego nie wie (bo jest to zbyt nowe lub spoza zakresu jego treningu), może **wygenerować odpowiedź, która brzmi prawdziwie, ale jest niepoprawna lub całkowicie zmyślona**. Nazywamy to **halucynacją**.

**Dlaczego to ważne?**

Jeżeli potrzebujemy odpowiedzi opartej na:
*   **Aktualnych informacjach** (np. dzisiejsze wiadomości, obecne trendy).
*   **Konkretnych, nowych danych** (np. treść właśnie opublikowanej strony internetowej, dane z najnowszego raportu).

Musimy **sami dostarczyć te informacje** modelowi w treści naszego zapytania (promptu).

W kolejnej komórce sprawdzimy, jak model reaguje na pytanie o wydarzenie spoza zakresu jego wiedzy.

In [9]:
import os
from openai import OpenAI, RateLimitError, APIError, AuthenticationError
import json

# Sprawdzenie, czy klucz API jest załadowany
openai_api_key = os.environ.get("OPENAI_API_KEY")

if not openai_api_key:
    print("❌ BŁĄD: Klucz OpenAI API nie został znaleziony w zmiennych środowiskowych.")
    print("Upewnij się, że uruchomiłeś komórkę konfiguracji kluczy API na początku notatnika.")
else:
    try:
        print("Inicjalizacja klienta OpenAI API...")
        client = OpenAI()

        # Definiujemy pytanie o przyszłe (lub bardzo niedawne/nieistniejące) wydarzenie
        prompt_o_przyszlosc = "Jakie jest bezrobocie w Polsce?"
        print(f"\nWysyłam zapytanie do modelu: '{prompt_o_przyszlosc}'")

        # Wybieramy model (można zmienić na inny, np. gpt-4o-mini)
        model_openai = "gpt-4o"
        print(f"Używany model: {model_openai}")

        # Wywołanie API
        completion = client.chat.completions.create(
            model=model_openai,
            messages=[
                {"role": "system", "content": "Odpowiadaj krótko i na temat."},
                {"role": "user", "content": prompt_o_przyszlosc}
            ],
            temperature=0.2, # Niższa temperatura dla bardziej przewidywalnej odpowiedzi
            max_tokens=100
        )

        # Przetwarzanie odpowiedzi
        if completion.choices:
            response_content = completion.choices[0].message.content
            print("\n--- Odpowiedź Modelu ---")
            print(response_content)
            print("-" * 25)
            print("Analiza: Zwróć uwagę, czy model przyznał, że nie ma tej informacji (poprawnie),")
            print("czy może próbował 'wymyślić' wynik (halucynacja).")
            if completion.usage:
                print(f"\nZużycie tokenów: {completion.usage}")
        else:
            print("⚠️ Nie otrzymano odpowiedzi (choices) od modelu.")

    # Obsługa błędów API
    except AuthenticationError:
        print("\n❌ BŁĄD API: Problem z uwierzytelnieniem. Sprawdź swój klucz API.")
    except RateLimitError:
        print("\n❌ BŁĄD API: Przekroczono limit żądań (Rate Limit). Spróbuj później.")
    except APIError as e:
        print(f"\n❌ BŁĄD API OpenAI: Status={e.status_code}, Treść={e.body}")
    except Exception as e:
        print(f"\n❌ Wystąpił nieoczekiwany błąd: {e}")

print("\n--- Koniec demonstracji ograniczonej wiedzy ---")

Inicjalizacja klienta OpenAI API...

Wysyłam zapytanie do modelu: 'Jakie jest bezrobocie w Polsce?'
Używany model: gpt-4o

--- Odpowiedź Modelu ---
Na koniec września 2023 roku stopa bezrobocia w Polsce wynosiła około 5,0%.
-------------------------
Analiza: Zwróć uwagę, czy model przyznał, że nie ma tej informacji (poprawnie),
czy może próbował 'wymyślić' wynik (halucynacja).

Zużycie tokenów: CompletionUsage(completion_tokens=27, prompt_tokens=33, total_tokens=60, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

--- Koniec demonstracji ograniczonej wiedzy ---


# 2. Źródło Danych: API Wyników Wyszukiwania (np. SerpData.io)

Aby dostarczyć modelowi językowemu aktualnych informacji możemy skorzystać z różnych API dostępnych na rynku. Np. takich do pobierania wyników wyszukiwania.

**Przykład: SerpData.io**

W tej lekcji jako przykład wykorzystamy API **SerpData.io**. Pozwala ono na:
*   Pobranie wyników wyszukiwania Google dla podanego **słowa kluczowego**.
*   Określenie **kraju** (`gl`) i **języka** (`hl`) wyszukiwania.
*   Uzyskanie odpowiedzi w formacie **JSON**, zawierającej m.in.:
    *   Listę **wyników organicznych** (`organic`) z tytułami, opisami i linkami.
    *   Ich **pozycję globalną** (`global_rank`) na stronie wyników.
    *   Inne elementy SERP (np. reklamy, mapy, "People Also Ask" - w zależności od API i zapytania).

**Wymagania:**
*   Podobnie jak API modeli językowych, takie usługi zazwyczaj wymagają **rejestracji** i uzyskania **klucza API** do uwierzytelniania zapytań.
*   Usługi te często mają **limity użycia** w planach darmowych lub są w pełni płatne.

W kolejnej komórce połączymy się z API SerpData.io, pobierzemy wyniki organiczne dla zadanego słowa kluczowego i zapiszemy wybrane dane do dalszej analizy.

In [10]:
import http.client
import json
import os
import urllib.parse
import pandas as pd
from IPython.display import display, Markdown, JSON

# --- Konfiguracja Użytkownika ---
#@markdown **Parametry Zapytania do API:**
keyword_to_search = "Ile wynosi bezrobocie w Polsce?" #@param {type:"string"}
language_code = "pl" #@param {type:"string"}
country_code = "pl" #@param {type:"string"}

#@markdown ---
#@markdown **Filtrowanie Wyników:**
#@markdown Zapiszemy linki i pozycje tylko dla TOP N wyników organicznych (wg pola 'rank'):
top_n_results = 10 #@param {type:"slider", min:1, max:50, step:1}
#@markdown ---

# --- Pobranie Klucza API ze Zmiennej Środowiskowej ---
serpdata_api_key = os.environ.get("SERPDATA_API_KEY")
# ----------------------------------------------------

# Zmienna globalna do przechowania wyników
serp_results_df = pd.DataFrame(columns=['Link', 'Rank'])
output_md_api = "" # Logi do wyświetlenia w Markdown

if not serpdata_api_key:
    output_md_api = "❌ **Brak klucza API SerpData.io w zmiennych środowiskowych.**\n"
    output_md_api += "Upewnij się, że dodałeś sekret `SECRET_SERPDATA_API_KEY` w Colab Secrets i uruchomiłeś pierwszą komórkę konfiguracji kluczy."
    display(Markdown(output_md_api))
    print("BŁĄD: Brak klucza API SerpData.io.")
else:
    try:
        output_md_api += f"Przygotowuję zapytanie do SerpData.io (Klucz API pobrany ze środowiska)...\n"
        conn = http.client.HTTPSConnection("api.serpdata.io")
        headers = {'Authorization': f"Bearer {serpdata_api_key}"}

        encoded_keyword = urllib.parse.quote_plus(keyword_to_search)
        path = f"/api/search?keyword={encoded_keyword}&hl={language_code}&gl={country_code}"

        output_md_api += f"Wysyłam żądanie GET do: `https://api.serpdata.io{path}`\n"
        print(f"Wysyłam żądanie GET do: https://api.serpdata.io{path}")

        conn.request("GET", path, headers=headers)
        res = conn.getresponse()
        data = res.read()
        status_code = res.status
        conn.close()

        output_md_api += f"Otrzymano odpowiedź ze statusem: `{status_code}`\n"
        print(f"Status odpowiedzi API: {status_code}")

        if status_code == 200:
            try:
                data_decoded = data.decode("utf-8")
                response_json = json.loads(data_decoded)
                output_md_api += f"✅ Odpowiedź API odebrana i sparsowana jako JSON.\n"

                nested_data = response_json.get('data', {}).get('data', {}).get('data', {})
                organic_results = nested_data.get('organic', [])

                filtered_data = []

                if organic_results:
                    output_md_api += f"Znaleziono {len(organic_results)} wyników organicznych. Filtruję TOP {top_n_results} wg pola `rank`...\n"
                    for result in organic_results:
                        if 'link' in result and 'rank' in result:
                            rank_value = result['rank']
                            if isinstance(rank_value, (int, float)) and rank_value <= top_n_results:
                                filtered_data.append({'Link': result['link'],'Rank': rank_value})
                        else:
                           print(f"[Ostrzeżenie] Pominięto wynik organiczny z powodu braku klucza 'link' lub 'rank': {result.get('title', 'Brak tytułu')}")

                    if filtered_data:
                        serp_results_df = pd.DataFrame(filtered_data).sort_values(by='Rank').reset_index(drop=True)
                        output_md_api += f"✅ Zapisano {len(serp_results_df)} wyników (TOP {top_n_results}) do DataFrame `serp_results_df`.\n"
                        print(f"\n--- TOP {top_n_results} Wyników Organicznych (DataFrame) ---")
                        print(serp_results_df.to_string())
                        print("--- Koniec DataFrame ---")
                    else:
                        output_md_api += f"⚠️ Nie znaleziono wyników organicznych w zakresie TOP {top_n_results}. DataFrame `serp_results_df` jest pusty.\n"
                        serp_results_df = pd.DataFrame(columns=['Link', 'Rank'])
                else:
                    output_md_api += f"⚠️ Odpowiedź API nie zawierała wyników organicznych w oczekiwanej ścieżce. DataFrame `serp_results_df` jest pusty.\n"
                    serp_results_df = pd.DataFrame(columns=['Link', 'Rank'])

            except json.JSONDecodeError as e:
                output_md_api += f"<span style='color:red;'>❌ Błąd podczas parsowania odpowiedzi JSON: {e}</span>\n"
                output_md_api += f"   Surowa odpowiedź (fragment): `{data_decoded[:500]}...`\n"
            except KeyError as e:
                output_md_api += f"<span style='color:red;'>❌ Błąd dostępu do klucza w odpowiedzi JSON: {e}. Sprawdź poprawność ścieżki dostępu.</span>\n"
            except Exception as e:
                 output_md_api += f"<span style='color:red;'>❌ Nieoczekiwany błąd podczas przetwarzania odpowiedzi: {e}</span>\n"

        # Obsługa błędów statusu - poprawione bloki try...except
        elif status_code == 401 or status_code == 403:
             output_md_api += f"<span style='color:red;'>❌ Błąd autoryzacji ({status_code}). Sprawdź poprawność klucza API SerpData.io.</span>\n"
             try:
                 output_md_api += f"   Odpowiedź serwera: `{data.decode('utf-8')}`\n"
             except Exception: # Jawnie łapiemy błąd dekodowania
                 output_md_api += "   (Nie można zdekodować odpowiedzi serwera)\n"
        elif status_code == 429:
             output_md_api += f"<span style='color:red;'>❌ Przekroczono limit zapytań (Rate Limit - {status_code}). Spróbuj później.</span>\n"
             try:
                 output_md_api += f"   Odpowiedź serwera: `{data.decode('utf-8')}`\n"
             except Exception:
                 output_md_api += "   (Nie można zdekodować odpowiedzi serwera)\n"
        else:
            output_md_api += f"<span style='color:red;'>❌ Otrzymano nieoczekiwany kod statusu: {status_code}.</span>\n"
            try:
                 output_md_api += f"   Odpowiedź serwera: `{data.decode('utf-8')}`\n"
            except Exception:
                 output_md_api += "   (Nie można zdekodować odpowiedzi serwera)\n"

    except http.client.HTTPException as e:
        output_md_api += f"<span style='color:red;'>❌ Błąd połączenia HTTP: {e}</span>\n"
        print(f"BŁĄD połączenia HTTP: {e}")
    except Exception as e:
        output_md_api += f"<span style='color:red;'>❌ Wystąpił nieoczekiwany błąd: {e}</span>\n"
        print(f"Nieoczekiwany błąd: {e}")

    # Wyświetlenie logów w Markdown pod komórką
    display(Markdown(output_md_api))

print(f"\n--- Koniec pobierania danych z API SERP ---")
# Zmienna globalna `serp_results_df` jest gotowa do użycia w następnych komórkach.

Wysyłam żądanie GET do: https://api.serpdata.io/api/search?keyword=Ile+wynosi+bezrobocie+w+Polsce%3F&hl=pl&gl=pl
Status odpowiedzi API: 200

--- TOP 10 Wyników Organicznych (DataFrame) ---
                                                                                                                                     Link  Rank
0                                                       https://kig.pl/aktualnosc-ekonomicz/bezrobocie-w-polsce-dane-gus-za-styczen-2025/     1
1  https://stat.gov.pl/obszary-tematyczne/rynek-pracy/bezrobocie-rejestrowane/stopa-bezrobocia-rejestrowanego-w-latach-1990-2025,4,1.html     2
2                                                       https://www.bankier.pl/gospodarka/wskazniki-makroekonomiczne/stopa-bezrobocia-pol     3
3                                                        https://kig.pl/aktualnosc-ekonomicz/bezrobocie-w-polsce-dane-gus-za-pazdziernik/     4
4                                               https://www.gov.pl/web/rodzina/bezrobocie-r

Przygotowuję zapytanie do SerpData.io (Klucz API pobrany ze środowiska)...
Wysyłam żądanie GET do: `https://api.serpdata.io/api/search?keyword=Ile+wynosi+bezrobocie+w+Polsce%3F&hl=pl&gl=pl`
Otrzymano odpowiedź ze statusem: `200`
✅ Odpowiedź API odebrana i sparsowana jako JSON.
Znaleziono 10 wyników organicznych. Filtruję TOP 10 wg pola `rank`...
✅ Zapisano 10 wyników (TOP 10) do DataFrame `serp_results_df`.



--- Koniec pobierania danych z API SERP ---


# 3. Źródło Danych: Pobieranie Treści Stron przez API (np. Jina AI Reader)

W poprzednim kroku uzyskaliśmy listę adresów URL z wyników wyszukiwania. Jednak sam link to za mało, aby model językowy mógł przeanalizować zawartość strony. Modele LLM nie mają wbudowanego dostępu do internetu, aby "odwiedzić" te linki.

**Problem:** Jak dostarczyć modelowi **treść** stron znajdujących się pod tymi adresami URL?

**Rozwiązanie:** Możemy użyć kolejnego zewnętrznego API, które specjalizuje się w pobieraniu i czyszczeniu treści ze stron internetowych. Jednym z takich narzędzi jest **Jina AI Reader API** (`r.jina.ai`).

**Co robi Jina AI Reader?**

*   Przyjmuje adres URL jako wejście.
*   Odwiedza stronę pod tym adresem.
*   Analizuje jej strukturę HTML.
*   **Wyodrębnia główną treść artykułu/strony**, starając się pominąć elementy nawigacyjne, reklamy, stopki itp. (działa jak tryb "Czytelnia" w przeglądarkach).
*   Zwraca oczyszczoną treść w wybranym formacie, np. jako **Markdown**, co jest bardzo wygodne do dalszego przetwarzania przez LLM.

**Dlaczego to przydatne?**

*   **Upraszcza Scraping:** Zamiast pisać własny, skomplikowany kod do scrapingu i czyszczenia HTML dla każdej strony, używamy gotowego API.
*   **Czyste Dane dla LLM:** Dostarcza modelowi tekst w czytelnym formacie (Markdown), pozbawiony zbędnego "szumu" HTML.
*   **Aktualność:** Pozwala zasilić LLM **aktualną treścią** analizowanych stron, a nie tylko ich tytułami czy opisami z SERP.

**Wymagania:**
*   Podobnie jak inne API, Jina AI Reader wymaga **klucza API** do uwierzytelnienia (powinien być już skonfigurowany w pierwszej komórce notatnika).
*   Usługa może mieć swoje limity użycia.

W kolejnej komórce użyjemy Jina AI Reader API, aby pobrać treść dla kilku pierwszych linków z naszego DataFrame `serp_results_df`.

In [11]:
import os
import requests # Użyjemy biblioteki requests dla prostoty
import pandas as pd
from IPython.display import display, Markdown

# --- Konfiguracja ---
#@markdown Ile pierwszych URLi z DataFrame przetworzyć?
urls_to_process = 3 #@param {type:"slider", min:1, max:10, step:1}

# --- Pobranie Klucza API i Sprawdzenie Danych Wejściowych ---
jina_api_key = os.environ.get("JINA_API_KEY")
output_md_jina = ""

# Sprawdzamy, czy DataFrame istnieje i ma dane
if 'serp_results_df' not in globals() or serp_results_df.empty:
    output_md_jina = "❌ **Błąd:** DataFrame `serp_results_df` nie istnieje lub jest pusty. Uruchom najpierw komórkę pobierającą dane z API SERP (komórka 2)."
elif not jina_api_key:
    output_md_jina = "❌ **Brak klucza Jina AI API w zmiennych środowiskowych.**\n"
    output_md_jina += "Upewnij się, że dodałeś sekret `SECRET_JINA_API_KEY` w Colab Secrets i uruchomiłeś pierwszą komórkę konfiguracji kluczy."
else:
    try:
        output_md_jina += f"Przygotowuję pobieranie treści dla {urls_to_process} pierwszych URLi za pomocą Jina AI Reader...\n"
        print(f"Przygotowuję pobieranie treści dla {urls_to_process} pierwszych URLi...")

        # Wybieramy N pierwszych URLi
        urls = serp_results_df['Link'].head(urls_to_process).tolist()
        indices_to_update = serp_results_df.head(urls_to_process).index # Zapamiętujemy indeksy do aktualizacji

        fetched_content_list = [] # Lista na pobrane treści

        # Definiujemy nagłówki dla Jina API
        headers = {
            'Authorization': f'Bearer {jina_api_key}',
            'Accept': 'application/json', # Można też prosić o JSON, ale Markdown jest często lepszy dla LLM
            'X-Return-Format': 'markdown' # Prosimy o format Markdown
        }
        base_jina_url = "https://r.jina.ai/"

        # Pętla przez wybrane URL-e
        for i, url in enumerate(urls):
            jina_request_url = base_jina_url + url
            output_md_jina += f"\n**Pobieranie ({i+1}/{urls_to_process}):** `{url}`\n"
            print(f"Pobieranie ({i+1}/{urls_to_process}): {url} ...")

            try:
                response = requests.get(jina_request_url, headers=headers, timeout=60) # Dodano timeout

                if response.status_code == 200:
                    content = response.text # Jina zwraca treść bezpośrednio w ciele odpowiedzi
                    fetched_content_list.append(content)
                    output_md_jina += f"<span style='color:green;'>✅ Sukces!</span> Pobrano treść (fragment):\n```markdown\n{content[:300]}...\n```\n"
                    print("   Sukces!")
                else:
                    error_message = f"BŁĄD: Status {response.status_code} - {response.text[:200]}"
                    fetched_content_list.append(error_message) # Zapisz błąd zamiast treści
                    output_md_jina += f"<span style='color:red;'>❌ Błąd: Status {response.status_code}</span> - {response.text[:200]}\n"
                    print(f"   Błąd: Status {response.status_code}")

            except requests.exceptions.Timeout:
                error_message = "BŁĄD: Timeout podczas połączenia z Jina AI."
                fetched_content_list.append(error_message)
                output_md_jina += f"<span style='color:red;'>❌ {error_message}</span>\n"
                print(f"   {error_message}")
            except requests.exceptions.RequestException as e:
                error_message = f"BŁĄD: Problem z połączeniem/żądaniem do Jina AI: {e}"
                fetched_content_list.append(error_message)
                output_md_jina += f"<span style='color:red;'>❌ {error_message}</span>\n"
                print(f"   {error_message}")

        # Dodanie nowej kolumny do oryginalnego DataFrame (lub utworzenie, jeśli nie istnieje)
        # Używamy .loc do przypisania wartości do odpowiednich wierszy
        if 'Fetched Content' not in serp_results_df.columns:
             serp_results_df['Fetched Content'] = pd.NA # Utwórz kolumnę z wartościami NA

        # Przypisz pobrane treści do odpowiednich wierszy za pomocą indeksów
        serp_results_df.loc[indices_to_update, 'Fetched Content'] = fetched_content_list

        output_md_jina += f"\n✅ Zakończono pobieranie. Dodano kolumnę `Fetched Content` do DataFrame `serp_results_df` dla przetworzonych wierszy.\n"
        print("\n--- Podgląd DataFrame z pobraną treścią (kolumny Link i Fetched Content) ---")
        # Wyświetlamy tylko relewantne kolumny i wiersze dla czytelności
        display(serp_results_df.loc[indices_to_update, ['Link', 'Fetched Content']])
        print("--- Koniec podglądu ---")


    except Exception as e:
        output_md_jina += f"\n❌ **Wystąpił nieoczekiwany błąd:**\n   `{e}`\n"
        print(f"Nieoczekiwany błąd: {e}")

# Wyświetlenie logów w Markdown pod komórką
display(Markdown(output_md_jina))

print(f"\n--- Koniec pobierania treści stron z Jina AI ---")
# Zmienna globalna `serp_results_df` została zaktualizowana o nową kolumnę.

Przygotowuję pobieranie treści dla 3 pierwszych URLi...
Pobieranie (1/3): https://kig.pl/aktualnosc-ekonomicz/bezrobocie-w-polsce-dane-gus-za-styczen-2025/ ...
   Sukces!
Pobieranie (2/3): https://stat.gov.pl/obszary-tematyczne/rynek-pracy/bezrobocie-rejestrowane/stopa-bezrobocia-rejestrowanego-w-latach-1990-2025,4,1.html ...
   Sukces!
Pobieranie (3/3): https://www.bankier.pl/gospodarka/wskazniki-makroekonomiczne/stopa-bezrobocia-pol ...
   Sukces!

--- Podgląd DataFrame z pobraną treścią (kolumny Link i Fetched Content) ---


,Link,Fetched Content
0,https://kig.pl/aktualnosc-ekonomicz/bezrobocie...,"{""code"":200,""status"":20000,""data"":{""title"":""Be..."
1,https://stat.gov.pl/obszary-tematyczne/rynek-p...,"{""code"":200,""status"":20000,""data"":{""title"":""St..."
2,https://www.bankier.pl/gospodarka/wskazniki-ma...,"{""code"":200,""status"":20000,""data"":{""title"":""Ws..."


--- Koniec podglądu ---


Przygotowuję pobieranie treści dla 3 pierwszych URLi za pomocą Jina AI Reader...

**Pobieranie (1/3):** `https://kig.pl/aktualnosc-ekonomicz/bezrobocie-w-polsce-dane-gus-za-styczen-2025/`
<span style='color:green;'>✅ Sukces!</span> Pobrano treść (fragment):
```markdown
{"code":200,"status":20000,"data":{"title":"Bezrobocie w Polsce: Dane GUS za styczeń 2025 - Krajowa Izba Gospodarcza","description":"Bezrobocie w grudniu 2024. Stopa bezrobocia wynosząc w końcu grudnia 5,1% była o 0,1 pkt proc. wyższa niż miesiąc wcześniej","url":"https://kig.pl/aktualnosc-ekonomicz...
```

**Pobieranie (2/3):** `https://stat.gov.pl/obszary-tematyczne/rynek-pracy/bezrobocie-rejestrowane/stopa-bezrobocia-rejestrowanego-w-latach-1990-2025,4,1.html`
<span style='color:green;'>✅ Sukces!</span> Pobrano treść (fragment):
```markdown
{"code":200,"status":20000,"data":{"title":"Stopa bezrobocia rejestrowanego w latach 1990-2025","description":"","url":"https://stat.gov.pl/obszary-tematyczne/rynek-pracy/bezrobocie-rejestrowane/stopa-bezrobocia-rejestrowanego-w-latach-1990-2025,4,1.html","content":"Główny Urząd Statystyczny / Obsza...
```

**Pobieranie (3/3):** `https://www.bankier.pl/gospodarka/wskazniki-makroekonomiczne/stopa-bezrobocia-pol`
<span style='color:green;'>✅ Sukces!</span> Pobrano treść (fragment):
```markdown
{"code":200,"status":20000,"data":{"title":"Wskaźniki makroekonomiczne - Gospodarka - Bankier.pl","description":"Wskaźniki makroekonomiczne z Polski i ze świata w portalu Bankier.pl. Inflacja, stopa bezrobocia, PKB oraz inne wskaźniki.","url":"https://www.bankier.pl/gospodarka/wskazniki-makroekonomi...
```

✅ Zakończono pobieranie. Dodano kolumnę `Fetched Content` do DataFrame `serp_results_df` dla przetworzonych wierszy.



--- Koniec pobierania treści stron z Jina AI ---


# 5. Wstępne Przetwarzanie Danych Przed Przekazaniem do LLM

Pobraliśmy treść z kilku stron internetowych za pomocą API Jina AI Reader. Mamy teraz surowy materiał tekstowy, który potencjalnie zawiera wartościowe informacje.

**Ważna Uwaga:** W rzeczywistych, bardziej złożonych zastosowaniach, **rzadko wysyłamy całą, surową treść strony bezpośrednio do finalnego promptu** modelu językowego. Zazwyczaj konieczne jest dodatkowe przetwarzanie, takie jak:

1.  **Dokładniejsze Czyszczenie:** Usuwanie pozostałych nieistotnych elementów, stopki, menu, reklam, które mogły nie zostać w pełni odfiltrowane przez API takie jak Jina.
2.  **Selekcja Fragmentów:** Wybieranie tylko tych sekcji tekstu, które są najbardziej relewantne dla naszego zadania (np. tylko główna część artykułu, specyficzne sekcje opisujące dany produkt).
3.  **Chunking (Dzielenie na Fragmenty):** Jeśli łączna treść z wielu stron (lub nawet jednej długiej strony) przekracza limit tokenów wejściowych modelu (Context Window), **musimy** podzielić ją na mniejsze fragmenty i przetwarzać je osobno lub w kilku krokach.
4.  **Wstępna Ekstrakcja/Podsumowanie:** Czasami bardziej efektywne jest najpierw poprosić LLM o wykonanie prostszego zadania na surowej treści (np. "Wyciągnij tylko kluczowe zdania dotyczące [temat]" lub "Podsumuj ten tekst do 200 słów"), a dopiero *wynik* tego działania przekazać do bardziej złożonego, finalnego promptu.

**W tej lekcji, dla uproszczenia demonstracji, pominiemy zaawansowane przetwarzanie i chunking.** Połączymy treść z *kilku pierwszych* pobranych stron i przekażemy ją do modelu z zadaniem ekstrakcji faktów. Pamiętaj jednak, że przy pracy z dłuższymi treściami lub większą liczbą źródeł, te kroki przetwarzania są niezbędne.

W kolejnej komórce wyślemy połączoną treść do API OpenAI z prośbą o ekstrakcję faktów.

In [14]:
import os
from openai import OpenAI, RateLimitError, APIError, AuthenticationError
import pandas as pd
import json
import io # Potrzebne do wczytania stringa jako pliku CSV
from IPython.display import display, Markdown

# --- Konfiguracja ---
#@markdown Ile pierwszych treści z DataFrame użyć do ekstrakcji?
sources_to_use = 3 #@param {type:"slider", min:1, max:5, step:1}

# --- Pobranie Klucza API i Danych Wejściowych ---
openai_api_key = os.environ.get("OPENAI_API_KEY")
output_md_extract = ""
model_openai_extract = "gpt-4o-mini"

# === Inicjalizacja DataFrame na wyniki ekstrakcji (teraz tylko 1 kolumna) ===
extracted_facts_df = pd.DataFrame(columns=['Fact']) # Zmieniono kolumnę na 'Fact'
# ========================================================================

# Sprawdzamy, czy DataFrame źródłowy istnieje
if 'serp_results_df' not in globals() or serp_results_df.empty:
    output_md_extract = "❌ **Błąd:** DataFrame `serp_results_df` nie istnieje lub jest pusty. Uruchom komórki 2 i 4."
elif 'Fetched Content' not in serp_results_df.columns:
     output_md_extract = "❌ **Błąd:** DataFrame `serp_results_df` nie zawiera kolumny 'Fetched Content'. Uruchom komórkę 4."
elif not openai_api_key:
    output_md_extract = "❌ **Brak klucza OpenAI API w zmiennych środowiskowych.**"
else:
    try:
        output_md_extract += f"Przygotowuję dane do ekstrakcji faktów z {sources_to_use} źródeł...\n"
        print(f"Przygotowuję dane z {sources_to_use} źródeł...")

        valid_content_list = []
        source_urls = []
        for index, row in serp_results_df.head(len(serp_results_df)).iterrows():
             if len(valid_content_list) >= sources_to_use: break
             content = row['Fetched Content']
             if pd.notna(content) and isinstance(content, str) and not content.startswith("BŁĄD:"):
                 valid_content_list.append(content)
                 source_urls.append(row['Link'])

        if not valid_content_list:
             output_md_extract += f"<span style='color:red;'>❌ Nie znaleziono wystarczającej liczby poprawnie pobranych treści.</span>\n"
        else:
             combined_content = ""
             output_md_extract += f"Znaleziono {len(valid_content_list)} poprawnych źródeł. Łączenie treści...\n"
             for i, content in enumerate(valid_content_list):
                 combined_content += f"\n\n--- ŹRÓDŁO {i+1} ({source_urls[i]}) ---\n\n"
                 combined_content += content

             # === Zaktualizowany Prompt Systemowy ===
             system_prompt_extract = """
#TASK

Using only the content supplied below (combined from web pages), extract every distinct fact—names, numbers, dates, places, statistics—exactly as written in the text.

OUTPUT (CSV, UTF-8)
Column header: Fact
Each row: one unique fact
Do not output anything except the CSV, including the header.

"""
             # ======================================

             user_prompt_extract = f"""--- START TEXT ---
{combined_content}
--- END TEXT ---"""

             output_md_extract += f"Tworzę prompt dla modelu `{model_openai_extract}` z instrukcją generowania CSV...\n"
             print(f"Wysyłam zapytanie do {model_openai_extract}...")
             output_md_extract += f"Wysyłam zapytanie do `{model_openai_extract}`...\n"
             client = OpenAI()
             completion = client.chat.completions.create(
                 model=model_openai_extract,
                 messages=[
                     {"role": "system", "content": system_prompt_extract},
                     {"role": "user", "content": user_prompt_extract}
                 ],
                 temperature=0.1, # Jeszcze niższa temperatura dla precyzyjnej ekstrakcji
                 max_tokens=5000 # Zwiększamy na wypadek wielu faktów
             )

             if completion.choices:
                 extracted_facts_csv_string = completion.choices[0].message.content
                 output_md_extract += f"\n**--- Odpowiedź Modelu (Oczekiwany Format CSV) ---**\n"
                 # Wyświetlamy surową odpowiedź jako blok kodu
                 display(Markdown(f"```csv\n{extracted_facts_csv_string}\n```"))
                 if completion.usage:
                      print(f"\nZużycie tokenów: {completion.usage}")

                 # --- Próba wczytania odpowiedzi jako CSV do DataFrame ---
                 print("\nPróbuję wczytać odpowiedź jako CSV do DataFrame `extracted_facts_df`...")
                 try:
                     # Używamy io.StringIO, aby traktować string jak plik
                     csv_file = io.StringIO(extracted_facts_csv_string)
                     # Wczytujemy CSV, zakładając, że pierwsza linia to nagłówek 'Fact'
                     temp_df = pd.read_csv(csv_file)

                     # Sprawdzamy, czy wczytano kolumnę 'Fact' (lub cokolwiek)
                     if not temp_df.empty and 'Fact' in temp_df.columns:
                          extracted_facts_df = temp_df[['Fact']] # Wybieramy tylko kolumnę 'Fact'
                          output_md_extract += f"\n✅ Pomyślnie wczytano CSV i zapisano {len(extracted_facts_df)} faktów do DataFrame `extracted_facts_df`.\n"
                          print("\n--- DataFrame z Wyekstrahowanymi Faktami (`extracted_facts_df`) ---")
                          display(extracted_facts_df)
                          print("--- Koniec DataFrame ---")
                     elif not temp_df.empty:
                          output_md_extract += f"<span style='color:orange;'>⚠️ Wczytano CSV, ale nie znaleziono kolumny 'Fact'. Użyto pierwszej kolumny.</span>\n"
                          # Bierzemy pierwszą kolumnę jako fakty
                          extracted_facts_df = temp_df.iloc[:, [0]]
                          extracted_facts_df.columns = ['Fact'] # Nadajemy nazwę 'Fact'
                          print("\n--- DataFrame z Wyekstrahowanymi Faktami (Pierwsza Kolumna) ---")
                          display(extracted_facts_df)
                          print("--- Koniec DataFrame ---")
                     else:
                          output_md_extract += f"<span style='color:orange;'>⚠️ Odpowiedź wyglądała jak CSV, ale nie udało się wczytać danych lub brak kolumny 'Fact'.</span>\n"
                          extracted_facts_df = pd.DataFrame(columns=['Fact']) # Zostawiamy pusty

                 except pd.errors.EmptyDataError:
                      output_md_extract += f"<span style='color:orange;'>⚠️ Odpowiedź modelu była pusta lub nie zawierała danych CSV.</span>\n"
                      extracted_facts_df = pd.DataFrame(columns=['Fact'])
                 except Exception as csv_e:
                      output_md_extract += f"<span style='color:red;'>❌ Błąd podczas wczytywania odpowiedzi jako CSV: {csv_e}. Sprawdź format odpowiedzi modelu.</span>\n"
                      extracted_facts_df = pd.DataFrame(columns=['Fact']) # Resetujemy DF w razie błędu
                 # ----------------------------------------------------

             else:
                 output_md_extract += "⚠️ Nie otrzymano odpowiedzi (choices) od modelu.\n"


    # ... (Reszta obsługi błędów bez zmian) ...
    except NameError as e:
         if 'keyword_to_search' in str(e): output_md_extract += f"<span style='color:red;'>❌ Błąd: Zmienna `keyword_to_search` nie jest zdefiniowana. Uruchom komórkę 2.</span>\n"
         else: output_md_extract += f"<span style='color:red;'>❌ Błąd zmiennej: {e}.</span>\n"
    except AuthenticationError: output_md_extract += f"\n❌ **BŁĄD API: Authentication Error**\n"
    except RateLimitError: output_md_extract += f"\n❌ **BŁĄD API: Rate Limit Error**\n"
    except APIError as e: output_md_extract += f"\n❌ **BŁĄD API OpenAI:** Status: `{e.status_code}`, Typ: `{e.type}`\n"
    except Exception as e: output_md_extract += f"\n❌ **Wystąpił nieoczekiwany błąd:** `{e}`\n"


# Wyświetlenie logów w Markdown pod komórką
display(Markdown(output_md_extract))

print(f"\n--- Koniec ekstrakcji faktów ---")
# Zmienna globalna `extracted_facts_df` jest teraz dostępna.

Przygotowuję dane z 3 źródeł...
Wysyłam zapytanie do gpt-4o-mini...


```csv
```csv
Fact
"Bezrobocie w grudniu 2024. Stopa bezrobocia wynosząc w końcu grudnia 5,1% była o 0,1 pkt proc. wyższa niż miesiąc wcześniej"
"Liczba zarejestrowanych bezrobotnych wyniosła 837,6 tys. osób"
"837,6 tys. osób była o 51,4 tys. osób wyższa niż w końcu grudnia"
"0,5 tys. osób wyższa niż przed rokiem"
"Styczniowy wzrost bezrobocia należy traktować jako sezonowy"
"W miesiącu tym bezrobocie ulega zwiększeniu o około 30-50 tys. osób"
"Liczba nowo zarejestrowanych bezrobotnych wyniosła 131,1 tys."
"131,1 tys. wobec 94,4 tys. w grudniu"
"134,7 tys. w styczniu roku 2024"
"W styczniu wyrejestrowano 79,6 tys. osób"
"79,6 tys. wobec 82,7 tys. w grudniu"
"85,8 tys. w styczniu roku 2024"
"Ofert pracy pojawiło się na przestrzeni stycznia 88,5 tys."
"88,5 tys. wobec 60,1 tys. w grudniu"
"99,1 tys. przed dwunastoma miesiącami"
"Stopa bezrobocia wynosząc w końcu stycznia 5,4%"
"5,4% była o 0,3 pkt proc. wyższa niż miesiąc wcześniej"
"Stopa bezrobocia okazała się o 0,1 pkt proc wyższa niż przed rokiem"
"Ostatni przypadek, kiedy stopa bezrobocia okazała się wyższa niż przed dwunastoma miesiącami notowany był w sierpniu 2021"
"Poziom stopy bezrobocia w styczniu był wyższy od naszej prognozy i średniej z prognoz rynkowych o 0,1 pkt proc."
"Piotr Soroczyński – Główny Ekonomista KIG"
"Liczba pracujących wynosząc około 14 674 tys. osób"
"14 674 tys. osób była w styczniu niższa niż przed rokiem o 284 tys."
"1,90% mniej niż przed rokiem"
"Aktywnych zawodowo było w końcu stycznia 15 511 tys. osób"
"15 511 tys. osób – około 283 tys. i 1,79% mniej niż przed rokiem"
"Stopa bezrobocia na poziomie 5,5% przy liczbie bezrobotnych 855 tys. osób"
"Latem stopa bezrobocia obniży się do 4,9% przy liczbie bezrobotnych 760 tys. osób"
"Dla grudnia 2025 oczekiwana jest stopa bezrobocia 5,0% przy liczbie bezrobotnych 770 tys. osób"
"Stan w końcu marca 2025 r."
"Stopa bezrobocia rejestrowanego w latach 1990-2025"
"5,4% (I 2025)"
"0,30 p.p."
"5,10% (XII 2024)"
"6,50% (II 2021)"
"4,80% (VIII 2022)"
"Data: 24.04.2025"
"Stopa bezrobocia w 2025: 5,4%"
"Stopa bezrobocia w 2024: 5,4%"
"Stopa bezrobocia w 2023: 5,5%"
"Stopa bezrobocia w 2022: 5,9%"
"Stopa bezrobocia w 2021: 6,5%"
"Stopa bezrobocia w 2020: 5,5%"
"Stopa bezrobocia w 2019: 6,1%"
"Stopa bezrobocia w 2018: 6,8%"
"Stopa bezrobocia w 2017: 8,5%"
"Stopa bezrobocia w 2016: 10,2%"
"Stopa bezrobocia w 2015: 11,9%"
"Stopa bezrobocia w 2014: 13,9%"
"Stopa bezrobocia w 2013: 14,2%"
"Stopa bezrobocia w 2012: 13,2%"
"Stopa bezrobocia w 2011: 13,1%"
"Stopa bezrobocia w 2010: 12,9%"
"Stopa bezrobocia w 2009: 10,4%"
"Stopa bezrobocia w 2008: 11,5%"
"Stopa bezrobocia w 2007: 15,1%"
"Stopa bezrobocia w 2006: 18,0%"
"Stopa bezrobocia w 2005: 19,4%"
"Stopa bezrobocia w 2004: 20,6%"
"Stopa bezrobocia w 2003: 20,6%"
"Stopa bezrobocia w 2002: 18,1%"
"Stopa bezrobocia w 2001: 15,7%"
"Stopa bezrobocia w 2000: 13,7%"
"Stopa bezrobocia w 1999: 11,4%"
"Stopa bezrobocia w 1998: 10,7%"
"Stopa bezrobocia w 1997: 13,1%"
"Stopa bezrobocia w 1996: 15,4%"
"Stopa bezrobocia w 1995: 16,1%"
"Stopa bezrobocia w 1994: 16,7%"
"Stopa bezrobocia w 1993: 14,2%"
"Stopa bezrobocia w 1992: 12,1%"
"Stopa bezrobocia w 1991: 6,6%"
"Stopa bezrobocia w 1990: 0,3%"
```
```


Zużycie tokenów: CompletionUsage(completion_tokens=1338, prompt_tokens=37571, total_tokens=38909, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=37504))

Próbuję wczytać odpowiedź jako CSV do DataFrame `extracted_facts_df`...

--- DataFrame z Wyekstrahowanymi Faktami (Pierwsza Kolumna) ---


,Fact
0,Fact
1,Bezrobocie w grudniu 2024. Stopa bezrobocia wy...
2,Liczba zarejestrowanych bezrobotnych wyniosła ...
3,"837,6 tys. osób była o 51,4 tys. osób wyższa n..."
4,"0,5 tys. osób wyższa niż przed rokiem"
...,...
70,"Stopa bezrobocia w 1993: 14,2%"
71,"Stopa bezrobocia w 1992: 12,1%"
72,"Stopa bezrobocia w 1991: 6,6%"
73,"Stopa bezrobocia w 1990: 0,3%"


--- Koniec DataFrame ---


Przygotowuję dane do ekstrakcji faktów z 3 źródeł...
Znaleziono 3 poprawnych źródeł. Łączenie treści...
Tworzę prompt dla modelu `gpt-4o-mini` z instrukcją generowania CSV...
Wysyłam zapytanie do `gpt-4o-mini`...

**--- Odpowiedź Modelu (Oczekiwany Format CSV) ---**
<span style='color:orange;'>⚠️ Wczytano CSV, ale nie znaleziono kolumny 'Fact'. Użyto pierwszej kolumny.</span>



--- Koniec ekstrakcji faktów ---


# 7. Zasilanie Modeli Pozyskanymi Danymi - Poprawa Odpowiedzi

W pierwszej komórce widzieliśmy, że model językowy, zapytany o aktualne dane (np. aktualną stopę bezrobocia), może mieć problem z odpowiedzią lub nawet "halucynować", ponieważ nie posiada tych informacji w swojej bazie wiedzy.

Teraz pokażemy, jak **dostarczenie modelowi wcześniej pozyskanych i wyekstrahowanych danych** pozwala mu udzielić **poprawnej i opartej na faktach odpowiedzi**, nawet jeśli dotyczy to informacji, których sam by nie znał.


In [16]:
#@markdown **Pytanie do modelu:**
pytanie_uzytkownika = "Jakie jest bezrobocie w Polsce kwietniu 2025?" #@param {type:"string"}
#@markdown **Instrukcja systemowa (System Prompt):**
#@markdown (Zmienna `{dzisiejsza_data}` zostanie wstawiona automatycznie)
system_prompt_template = "Jesteś pomocnym asystentem. Odpowiadaj na pytania użytkownika wyłącznie na podstawie dostarczonego kontekstu. Dzisiejsza data to {dzisiejsza_data}." #@param {type:"string"}
#@markdown ---


import os
from openai import OpenAI, RateLimitError, APIError, AuthenticationError
import pandas as pd
import json
from datetime import date
from IPython.display import display, Markdown

# --- Pobranie Klucza API i Danych Wejściowych ---
openai_api_key = os.environ.get("OPENAI_API_KEY")
output_md_rag = ""
model_openai_rag = "gpt-4o-mini"

# --- Sprawdzenie Dostępności Danych ---
if 'extracted_facts_df' not in globals() or extracted_facts_df.empty:
    output_md_rag = "❌ **Błąd:** DataFrame `extracted_facts_df` nie istnieje lub jest pusty. Uruchom najpierw komórkę ekstrakcji faktów (komórka 6)."
elif not openai_api_key:
    output_md_rag = "❌ **Brak klucza OpenAI API w zmiennych środowiskowych.**"
elif not pytanie_uzytkownika:
     output_md_rag = "❌ **Błąd:** Pole 'Pytanie do modelu' nie może być puste."
else:
    try:
        output_md_rag += f"Przygotowuję wzbogacony prompt dla modelu `{model_openai_rag}`...\n"
        print(f"Przygotowuję wzbogacony prompt dla {model_openai_rag}...")

        # --- Krok 2: Dodanie Aktualnego Kontekstu (Data) ---
        dzisiejsza_data = date.today().strftime("%Y-%m-%d")
        # Formatujemy instrukcję systemową, wstawiając datę
        system_prompt_rag = system_prompt_template.format(dzisiejsza_data=dzisiejsza_data)
        output_md_rag += f"Instrukcja systemowa (po wstawieniu daty): `{system_prompt_rag}`\n"


        # --- Przygotowanie Kontekstu Faktów ---
        facts_context = ""
        if not extracted_facts_df.empty:
            if 'Fact' in extracted_facts_df.columns and len(extracted_facts_df.columns) == 1:
                 facts_context += "Oto wyekstrahowane fakty:\n"
                 for fact in extracted_facts_df['Fact'].tolist(): facts_context += f"- {fact}\n"
            else: # Fallback dla starszej wersji DF
                 facts_context += "Oto wyekstrahowane dane:\n"
                 fact_row = extracted_facts_df.iloc[0]
                 for header, value in fact_row.items(): facts_context += f"- {header}: {value}\n"
        else:
            facts_context = "Nie udało się wyekstrahować konkretnych faktów w poprzednim kroku."

        output_md_rag += f"\n**Kontekst (fakty) przekazywany do modelu:**\n```text\n{facts_context}\n```\n"

        # --- Krok 3: Budowa Promptu Użytkownika (z użyciem danych z formularza) ---
        user_prompt_rag = f"""Na podstawie poniższego kontekstu odpowiedz na pytanie:

Pytanie: {pytanie_uzytkownika}

Kontekst (fakty wyekstrahowane z różnych źródeł):
{facts_context}

Odpowiedz krótko i precyzyjnie, bazując tylko na informacjach z podanego kontekstu."""

        # --- Krok 4: Wysyłanie Wzbogaconego Zapytania ---
        print(f"Wysyłam wzbogacone zapytanie do {model_openai_rag}...")
        output_md_rag += f"\n**--- Zapytanie do Modelu (z Kontekstem) ---**\n"
        output_md_rag += f"Pytanie: `{pytanie_uzytkownika}`\n"

        client = OpenAI()
        completion_rag = client.chat.completions.create(
            model=model_openai_rag,
            messages=[
                {"role": "system", "content": system_prompt_rag}, # Używamy sformatowanego promptu systemowego
                {"role": "user", "content": user_prompt_rag}
            ],
            temperature=0.1,
            max_tokens=150
        )

        # --- Krok 5: Przetwarzanie i Porównanie Odpowiedzi ---
        output_md_rag += f"\n**--- Odpowiedź Modelu (z Kontekstem) ---**\n"
        if completion_rag.choices:
            response_content_rag = completion_rag.choices[0].message.content
            output_md_rag += f"\n```text\n{response_content_rag}\n```\n"

            if completion_rag.usage:
                 print(f"\nZużycie tokenów (RAG): {completion_rag.usage}")

            output_md_rag += f"\n**Porównanie:**\n"
            output_md_rag += f"Teraz porównaj tę odpowiedź z odpowiedzią uzyskaną w **komórce 1**, gdzie model nie miał dostępu do tych faktów. Powinieneś zauważyć znaczną różnicę w jakości i konkretności informacji.\n"
            output_md_rag += f"Dostarczenie kontekstu (technika RAG) pozwala modelowi odpowiadać na pytania dotyczące aktualnych lub specyficznych danych, unikając halucynacji.\n"
        else:
            output_md_rag += "⚠️ Nie otrzymano odpowiedzi (choices) od modelu dla zapytania RAG.\n"


    # ... (Reszta obsługi błędów bez zmian) ...
    except NameError as e:
         if 'extracted_facts_df' in str(e): output_md_rag += f"<span style='color:red;'>❌ Błąd: DataFrame `extracted_facts_df` nie jest zdefiniowany. Uruchom komórkę 6.</span>\n"
         elif 'keyword_to_search' in str(e): output_md_rag += f"<span style='color:orange;'>⚠️ Ostrzeżenie: Zmienna `keyword_to_search` nie jest zdefiniowana (komórka 2 nie uruchomiona?), użyto domyślnego tekstu w prompcie.</span>\n"
         else: output_md_rag += f"<span style='color:red;'>❌ Błąd zmiennej: {e}.</span>\n"
    except AuthenticationError: output_md_rag += f"\n❌ **BŁĄD API: Authentication Error**\n"
    except RateLimitError: output_md_rag += f"\n❌ **BŁĄD API: Rate Limit Error**\n"
    except APIError as e: output_md_rag += f"\n❌ **BŁĄD API OpenAI:** Status: `{e.status_code}`, Typ: `{e.type}`\n"
    except Exception as e: output_md_rag += f"\n❌ **Wystąpił nieoczekiwany błąd:** `{e}`\n"


# Wyświetlenie logów i podsumowania w Markdown pod komórką
display(Markdown(output_md_rag))

print(f"\n--- Koniec demonstracji odpowiedzi wzbogaconej (RAG) ---")

Przygotowuję wzbogacony prompt dla gpt-4o-mini...
Wysyłam wzbogacone zapytanie do gpt-4o-mini...

Zużycie tokenów (RAG): CompletionUsage(completion_tokens=23, prompt_tokens=1513, total_tokens=1536, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


Przygotowuję wzbogacony prompt dla modelu `gpt-4o-mini`...
Instrukcja systemowa (po wstawieniu daty): `Jesteś pomocnym asystentem. Odpowiadaj na pytania użytkownika wyłącznie na podstawie dostarczonego kontekstu. Dzisiejsza data to 2025-05-02.`

**Kontekst (fakty) przekazywany do modelu:**
```text
Oto wyekstrahowane fakty:
- Fact
- Bezrobocie w grudniu 2024. Stopa bezrobocia wynosząc w końcu grudnia 5,1% była o 0,1 pkt proc. wyższa niż miesiąc wcześniej
- Liczba zarejestrowanych bezrobotnych wyniosła 837,6 tys. osób
- 837,6 tys. osób była o 51,4 tys. osób wyższa niż w końcu grudnia
- 0,5 tys. osób wyższa niż przed rokiem
- Styczniowy wzrost bezrobocia należy traktować jako sezonowy
- W miesiącu tym bezrobocie ulega zwiększeniu o około 30-50 tys. osób
- Liczba nowo zarejestrowanych bezrobotnych wyniosła 131,1 tys.
- 131,1 tys. wobec 94,4 tys. w grudniu
- 134,7 tys. w styczniu roku 2024
- W styczniu wyrejestrowano 79,6 tys. osób
- 79,6 tys. wobec 82,7 tys. w grudniu
- 85,8 tys. w styczniu roku 2024
- Ofert pracy pojawiło się na przestrzeni stycznia 88,5 tys.
- 88,5 tys. wobec 60,1 tys. w grudniu
- 99,1 tys. przed dwunastoma miesiącami
- Stopa bezrobocia wynosząc w końcu stycznia 5,4%
- 5,4% była o 0,3 pkt proc. wyższa niż miesiąc wcześniej
- Stopa bezrobocia okazała się o 0,1 pkt proc wyższa niż przed rokiem
- Ostatni przypadek, kiedy stopa bezrobocia okazała się wyższa niż przed dwunastoma miesiącami notowany był w sierpniu 2021
- Poziom stopy bezrobocia w styczniu był wyższy od naszej prognozy i średniej z prognoz rynkowych o 0,1 pkt proc.
- Piotr Soroczyński – Główny Ekonomista KIG
- Liczba pracujących wynosząc około 14 674 tys. osób
- 14 674 tys. osób była w styczniu niższa niż przed rokiem o 284 tys.
- 1,90% mniej niż przed rokiem
- Aktywnych zawodowo było w końcu stycznia 15 511 tys. osób
- 15 511 tys. osób – około 283 tys. i 1,79% mniej niż przed rokiem
- Stopa bezrobocia na poziomie 5,5% przy liczbie bezrobotnych 855 tys. osób
- Latem stopa bezrobocia obniży się do 4,9% przy liczbie bezrobotnych 760 tys. osób
- Dla grudnia 2025 oczekiwana jest stopa bezrobocia 5,0% przy liczbie bezrobotnych 770 tys. osób
- Stan w końcu marca 2025 r.
- Stopa bezrobocia rejestrowanego w latach 1990-2025
- 5,4% (I 2025)
- 0,30 p.p.
- 5,10% (XII 2024)
- 6,50% (II 2021)
- 4,80% (VIII 2022)
- Data: 24.04.2025
- Stopa bezrobocia w 2025: 5,4%
- Stopa bezrobocia w 2024: 5,4%
- Stopa bezrobocia w 2023: 5,5%
- Stopa bezrobocia w 2022: 5,9%
- Stopa bezrobocia w 2021: 6,5%
- Stopa bezrobocia w 2020: 5,5%
- Stopa bezrobocia w 2019: 6,1%
- Stopa bezrobocia w 2018: 6,8%
- Stopa bezrobocia w 2017: 8,5%
- Stopa bezrobocia w 2016: 10,2%
- Stopa bezrobocia w 2015: 11,9%
- Stopa bezrobocia w 2014: 13,9%
- Stopa bezrobocia w 2013: 14,2%
- Stopa bezrobocia w 2012: 13,2%
- Stopa bezrobocia w 2011: 13,1%
- Stopa bezrobocia w 2010: 12,9%
- Stopa bezrobocia w 2009: 10,4%
- Stopa bezrobocia w 2008: 11,5%
- Stopa bezrobocia w 2007: 15,1%
- Stopa bezrobocia w 2006: 18,0%
- Stopa bezrobocia w 2005: 19,4%
- Stopa bezrobocia w 2004: 20,6%
- Stopa bezrobocia w 2003: 20,6%
- Stopa bezrobocia w 2002: 18,1%
- Stopa bezrobocia w 2001: 15,7%
- Stopa bezrobocia w 2000: 13,7%
- Stopa bezrobocia w 1999: 11,4%
- Stopa bezrobocia w 1998: 10,7%
- Stopa bezrobocia w 1997: 13,1%
- Stopa bezrobocia w 1996: 15,4%
- Stopa bezrobocia w 1995: 16,1%
- Stopa bezrobocia w 1994: 16,7%
- Stopa bezrobocia w 1993: 14,2%
- Stopa bezrobocia w 1992: 12,1%
- Stopa bezrobocia w 1991: 6,6%
- Stopa bezrobocia w 1990: 0,3%
- ```

```

**--- Zapytanie do Modelu (z Kontekstem) ---**
Pytanie: `Jakie jest bezrobocie w Polsce kwietniu 2025?`

**--- Odpowiedź Modelu (z Kontekstem) ---**

```text
W kwietniu 2025 stopa bezrobocia w Polsce wynosiła 5,4%.
```

**Porównanie:**
Teraz porównaj tę odpowiedź z odpowiedzią uzyskaną w **komórce 1**, gdzie model nie miał dostępu do tych faktów. Powinieneś zauważyć znaczną różnicę w jakości i konkretności informacji.
Dostarczenie kontekstu (technika RAG) pozwala modelowi odpowiadać na pytania dotyczące aktualnych lub specyficznych danych, unikając halucynacji.



--- Koniec demonstracji odpowiedzi wzbogaconej (RAG) ---


# 8. Optymalizacja Tokenów: Dlaczego Markdown jest często Lepszy niż HTML? 🤔

Pracując z API modeli językowych, często będziemy przetwarzać treści pobrane z internetu, na przykład artykuły z blogów czy opisy produktów. Te treści są zazwyczaj dostępne w formacie **HTML** (HyperText Markup Language), który oprócz samej treści zawiera mnóstwo znaczników strukturalnych, stylów, skryptów i metadanych.

**Problem z HTML w kontekście LLM:**

Cały ten dodatkowy "balast" w kodzie HTML, choć niezbędny do poprawnego wyświetlania strony w przeglądarce, jest często **niepotrzebny (a nawet szkodliwy)**, gdy chcemy, aby model językowy zrozumiał *znaczenie* tekstu. Co więcej, te wszystkie dodatkowe znaczniki **znacznie zwiększają liczbę tokenów** potrzebnych do reprezentacji tej samej treści w porównaniu do czystszego formatu.

**Rozwiązanie: Konwersja do Markdown**

**Markdown** to lekki język znaczników, który pozwala formatować tekst w prosty i czytelny sposób, używając minimalnej liczby znaków specjalnych (np. `*` dla kursywy, `**` dla pogrubienia, `#` dla nagłówków).

Konwertując pobrany HTML do formatu Markdown przed wysłaniem go do API LLM, możemy uzyskać znaczące korzyści:

1.  **Mniejsza Liczba Tokenów:** Usunięcie zbędnych znaczników HTML drastycznie redukuje liczbę tokenów, co oznacza:
    *   **Niższe koszty:** Płacisz mniej za tokeny wejściowe.
    *   **Więcej miejsca w kontekście:** Możesz zmieścić więcej *rzeczywistej* treści w limicie tokenów wejściowych modelu.
2.  **Lepsze Zrozumienie przez Model:** Czysty tekst w Markdown jest często łatwiejszy do przetworzenia i zrozumienia przez model niż "zaszumiony" kod HTML. Model może lepiej skupić się na semantyce treści.

**Demonstracja:**

W kolejnej komórce pokażemy praktyczny przykład. Pobierzemy zawartość strony internetowej (jej kod HTML), przekonwertujemy ją do formatu Markdown za pomocą biblioteki `markdownify`, a następnie porównamy liczbę tokenów dla obu wersji (surowego HTML i wygenerowanego Markdown) używając tokenizera `tiktoken`. To wyraźnie pokaże, jak duża może być różnica.

In [17]:
# --- Install necessary library ---
!pip install tiktoken requests -q

# --- Importy i Konfiguracja ---
import os
import requests
import tiktoken
import json
from IPython.display import display, HTML, Markdown
import logging
import html # For escaping HTML for display

# --- Basic Logging Setup ---
# Use INFO for standard messages, DEBUG for raw responses/details
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# --- Formularz ---
#@markdown **Podaj adres URL strony do analizy:**
target_url_jina = "https://www.sensai.academy/" #@param {type:"string"}
#@markdown ---
#@markdown **Fragmenty do wyświetlenia (liczba znaków):**
snippet_length_jina = 300 #@param {type:"integer"}
#@markdown ---
#@markdown **Koszt za 1000 tokenów (w USD):**
cost_per_1k_tokens = 0.2 #@param {type:"number"} # Example: Use OpenAI's gpt-3.5-turbo-0125 input pricing ($0.0005/1k) or adjust as needed
#@markdown ---


jina_api_key = os.environ.get("JINA_API_KEY") # Make sure this is set in Colab Secrets!
# If you don't have it as a secret, you can temporarily set it here for testing:
# jina_api_key = "YOUR_JINA_API_KEY_HERE" # <<< ONLY FOR TESTING, REMOVE/USE SECRETS LATER

base_jina_url = "https://r.jina.ai/"
html_content = None
markdown_content = None
html_tokens = -1
markdown_tokens = -1

# --- Logika Główna ---
if not jina_api_key:
    logging.error("❌ Brak klucza Jina AI API (JINA_API_KEY). Skonfiguruj go w Colab Secrets.")
else:
    try:
        # Przygotowanie tokenizera
        logging.info("Inicjalizacja tokenizera tiktoken (cl100k_base)...")
        # Use a common encoding, cl100k_base is good for newer OpenAI models
        encoding = tiktoken.get_encoding("cl100k_base")
        logging.info("Tokenizer gotowy.")

        jina_request_url = f"{base_jina_url}{target_url_jina}"
        request_timeout = 60 # Increased timeout slightly more

        # --- Krok 1: Pobranie treści jako Markdown (Oczekiwany format: JSON) ---
        logging.info(f"\n[1/5] Pobieranie treści jako MARKDOWN z Jina AI dla: {target_url_jina}")
        headers_md = {
            'Authorization': f'Bearer {jina_api_key}',
            'Accept': 'application/json', # Still expect JSON for Markdown
            'X-Return-Format': 'markdown'
        }
        try:
            response_md = requests.get(jina_request_url, headers=headers_md, timeout=request_timeout)
            response_md.raise_for_status() # Check for HTTP errors first
            response_data_md = response_md.json() # Parse JSON for Markdown

            if isinstance(response_data_md, dict) and 'data' in response_data_md and isinstance(response_data_md['data'], dict) and 'content' in response_data_md['data']:
                 markdown_content = response_data_md['data']['content']
                 if isinstance(markdown_content, str) and markdown_content:
                     logging.info("✅ Pobrano treść w formacie Markdown (z JSON).")
                     markdown_tokens = len(encoding.encode(markdown_content))
                     logging.info(f"   -> Liczba tokenów (Markdown): {markdown_tokens}")
                 else:
                     logging.warning("❌ Znaleziono 'data.content' dla Markdown, ale jest puste lub nie jest stringiem.")
                     logging.debug(f"   -> Typ: {type(markdown_content)}, Wartość (fragment): {repr(markdown_content)[:200]}...")
                     markdown_content = None # Mark as failed
            else:
                 logging.warning("❌ Nie znaleziono oczekiwanej struktury {'data': {'content': ...}} w odpowiedzi JSON dla Markdown.")
                 logging.debug(f"   Odpowiedź Jina (Markdown):\n{json.dumps(response_data_md, indent=2)}")
                 markdown_content = None # Mark as failed

        except requests.exceptions.Timeout:
             logging.error(f"❌ Timeout ({request_timeout}s) podczas pobierania Markdown.")
        except requests.exceptions.HTTPError as e_http:
             logging.error(f"❌ Błąd HTTP {e_http.response.status_code} podczas pobierania Markdown: {e_http}")
             logging.debug(f"   Odpowiedź serwera (Markdown): {e_http.response.text[:500]}...")
        except requests.exceptions.RequestException as e_req:
            logging.error(f"❌ Błąd sieciowy podczas pobierania Markdown: {e_req}")
        except json.JSONDecodeError as e_json:
            logging.error(f"❌ Błąd dekodowania JSON odpowiedzi dla Markdown: {e_json}")
            logging.debug(f"   Status code: {response_md.status_code}")
            logging.debug(f"   Surowa odpowiedź (Markdown):\n{response_md.text[:500]}...")
        except Exception as e:
            logging.error(f"❌ Nieoczekiwany błąd podczas pobierania Markdown: {e}", exc_info=True)


        # --- Krok 2: Pobranie treści jako HTML (Oczekiwany format: Raw HTML) ---
        logging.info(f"\n[2/5] Pobieranie treści jako HTML z Jina AI dla: {target_url_jina}")
        headers_html = {
            'Authorization': f'Bearer {jina_api_key}',
            # Accept header might be ignored if it sends raw HTML, but set anyway
            'Accept': 'text/html,*/*',
            'X-Return-Format': 'html'
        }
        try:
            response_html = requests.get(jina_request_url, headers=headers_html, timeout=request_timeout)
            response_html.raise_for_status() # Check for HTTP errors (4xx, 5xx)

            # --- MODIFIED LOGIC: Assume raw HTML in response.text ---
            # No JSON parsing here based on user feedback
            html_content = response_html.text

            if html_content and isinstance(html_content, str):
                logging.info("✅ Pobrano treść jako Raw HTML (bezpośrednio z odpowiedzi).")
                html_tokens = len(encoding.encode(html_content))
                logging.info(f"   -> Liczba tokenów (HTML): {html_tokens}")
            else:
                logging.warning("❌ Odpowiedź HTML była pusta lub niepoprawna.")
                html_content = None # Mark as failed
            # --- End of MODIFIED LOGIC ---

        except requests.exceptions.Timeout:
             logging.error(f"❌ Timeout ({request_timeout}s) podczas pobierania HTML.")
        except requests.exceptions.HTTPError as e_http:
             # This will catch 4xx/5xx errors even if the body isn't JSON
             logging.error(f"❌ Błąd HTTP {e_http.response.status_code} podczas pobierania HTML: {e_http}")
             logging.debug(f"   Odpowiedź serwera (HTML): {e_http.response.text[:500]}...")
             html_content = None # Mark as failed
        except requests.exceptions.RequestException as e_req:
            logging.error(f"❌ Błąd sieciowy podczas pobierania HTML: {e_req}")
            html_content = None # Mark as failed
        # We removed the JSONDecodeError handler here as we don't expect JSON
        except Exception as e:
            logging.error(f"❌ Nieoczekiwany błąd podczas przetwarzania odpowiedzi HTML: {e}", exc_info=True)
            html_content = None # Mark as failed


        # --- Krok 3: Porównanie Wyników ---
        print("\n[3/5] Porównywanie wyników...")
        print("\n--- Porównanie Liczby Tokenów (Tokenizer: cl100k_base) ---")

        valid_markdown = markdown_content is not None and markdown_tokens >= 0
        valid_html = html_content is not None and html_tokens >= 0

        if not valid_markdown:
             print("⚠️ Nie udało się poprawnie pobrać lub stokenizować wersji Markdown.")
        if not valid_html:
             print("⚠️ Nie udało się poprawnie pobrać lub stokenizować wersji HTML.")

        print(f"+-----------------+----------------+")
        print(f"| Format          | Liczba Tokenów |")
        print(f"+-----------------+----------------+")
        if valid_html:
            print(f"| Surowy HTML     | {html_tokens:<14} |")
        else:
            print(f"| Surowy HTML     | BŁĄD/BRAK      |")
        if valid_markdown:
            print(f"| Markdown        | {markdown_tokens:<14} |")
        else:
            print(f"| Markdown        | BŁĄD/BRAK      |")
        print(f"+-----------------+----------------+")


        if valid_markdown and valid_html:
            if html_tokens > 0: # Avoid division by zero
                 reduction_percent = ((html_tokens - markdown_tokens) / html_tokens) * 100
                 print(f"\n**Redukcja liczby tokenów (Markdown vs HTML): {reduction_percent:.2f}%**")
                 print("\n**Wnioski:** Użycie formatu Markdown zwróconego przez Jina AI Reader znacząco zmniejsza liczbę tokenów,")
                 print("co jest korzystne dla kosztów API i limitów kontekstu modeli LLM.")
            else:
                 print("\nNie można obliczyć redukcji procentowej (liczba tokenów HTML wynosi 0 lub mniej).")
        else:
             print("\nNie można przeprowadzić pełnego porównania redukcji z powodu błędów pobierania.")

        # --- Krok 4: Obliczenie Szacunkowych Kosztów ---
        print("\n[4/5] Obliczanie szacunkowych kosztów...")
        print(f"\n--- Szacunkowy Koszt Tokenów (przy {cost_per_1k_tokens:.6f} USD / 1k tokenów) ---")
        print(f"+-----------------+----------------+-----------------+")
        print(f"| Format          | Liczba Tokenów | Szacowany Koszt |")
        print(f"+-----------------+----------------+-----------------+")

        cost_html_str = "BŁĄD/BRAK"
        if valid_html:
            cost_html = (html_tokens / 1000) * cost_per_1k_tokens
            cost_html_str = f"${cost_html:.6f}"
            print(f"| Surowy HTML     | {html_tokens:<14} | {cost_html_str:<15} |")
        else:
            print(f"| Surowy HTML     | BŁĄD/BRAK      | {cost_html_str:<15} |")

        cost_markdown_str = "BŁĄD/BRAK"
        if valid_markdown:
            cost_markdown = (markdown_tokens / 1000) * cost_per_1k_tokens
            cost_markdown_str = f"${cost_markdown:.6f}"
            print(f"| Markdown        | {markdown_tokens:<14} | {cost_markdown_str:<15} |")
        else:
             print(f"| Markdown        | BŁĄD/BRAK      | {cost_markdown_str:<15} |")

        print(f"+-----------------+----------------+-----------------+")

        if valid_markdown and valid_html and cost_html > 0:
             savings_percent = ((cost_html - cost_markdown) / cost_html) * 100
             print(f"\n**Oszczędność kosztów dzięki Markdown: {savings_percent:.2f}%**")
        elif valid_markdown and valid_html:
             print("\nMarkdown jest potencjalnie tańszy, ale koszt HTML wynosi 0.")
        else:
            print("\nNie można obliczyć oszczędności z powodu błędów pobierania.")


        # --- Krok 5: Wyświetlenie Fragmentów ---
        print("\n[5/5] Wyświetlanie fragmentów...")

        if html_content:
            print(f"\n--- Fragment Surowego HTML (pierwsze {snippet_length_jina} znaków) ---")
            # Escape HTML for safe printing as code/text
            escaped_html_snippet = html.escape(html_content[:snippet_length_jina])
            print(escaped_html_snippet + "...")
            # Alternatively, to attempt rendering (might break layout or be huge):
            # display(HTML(html_content[:snippet_length_jina] + "..."))
        else:
            print("\n--- Fragment Surowego HTML ---")
            print("(Brak danych lub błąd pobierania)")

        if markdown_content:
            print(f"\n--- Fragment Wygenerowanego Markdown (pierwsze {snippet_length_jina} znaków) ---")
            # Display Markdown using IPython's Markdown renderer
            display(Markdown(markdown_content[:snippet_length_jina] + "..."))
            # Or just print raw markdown:
            # print(markdown_content[:snippet_length_jina] + "...")
        else:
             print("\n--- Fragment Wygenerowanego Markdown ---")
             print("(Brak danych lub błąd pobierania)")


    except tiktoken.EncodingNotFound as e_tik:
         logging.error(f"❌ Błąd inicjalizacji tokenizera tiktoken: {e_tik}")
    except Exception as e_main:
        logging.error(f"\n❌ Wystąpił nieoczekiwany błąd główny: {e_main}", exc_info=True)

print("\n--- Koniec Porównania HTML vs Markdown (Jina AI) ---")


[3/5] Porównywanie wyników...

--- Porównanie Liczby Tokenów (Tokenizer: cl100k_base) ---
+-----------------+----------------+
| Format          | Liczba Tokenów |
+-----------------+----------------+
| Surowy HTML     | 53029          |
| Markdown        | 20833          |
+-----------------+----------------+

**Redukcja liczby tokenów (Markdown vs HTML): 60.71%**

**Wnioski:** Użycie formatu Markdown zwróconego przez Jina AI Reader znacząco zmniejsza liczbę tokenów,
co jest korzystne dla kosztów API i limitów kontekstu modeli LLM.

[4/5] Obliczanie szacunkowych kosztów...

--- Szacunkowy Koszt Tokenów (przy 0.200000 USD / 1k tokenów) ---
+-----------------+----------------+-----------------+
| Format          | Liczba Tokenów | Szacowany Koszt |
+-----------------+----------------+-----------------+
| Surowy HTML     | 53029          | $10.605800      |
| Markdown        | 20833          | $4.166600       |
+-----------------+----------------+-----------------+

**Oszczędność kosztó

SensAI Academy - Program transformacji kariery - Naucz się SEO 3.0
===============
          

[Śpiesz się! Startujemy 12 maja Cena wzrośnie za: 2 dni i 11 : 04 : 59](https://www.sensai.academy/old-home-2)

[![Image 1: SENSAI logo](https://cdn.prod.website-files.com/66262ed45c87cfbe328a1e15/663aa040...


--- Koniec Porównania HTML vs Markdown (Jina AI) ---
